In [97]:
# %%
import numpy as np
from numpy.fft import rfftn, irfftn, rfftfreq, fftfreq
import matplotlib.pyplot as plt
import vtk
from noises import pink_noise, brownian_noise, white_noise

np.random.seed(42)

# Define the grid dimensions
nx = 64  # Number of points in x direction
ny = 64  # Number of points in y direction
nz = 64  # Number of points in z direction

# Define the range for x, y and z
x_min, x_max = 0, 1  # Range for x
y_min, y_max = 0, 1  # Range for y
z_min, z_max = 0, 1  # Range for z

# Create the vertices
x_vertices = np.linspace(x_min, x_max, nx+1)  # Create x values
y_vertices = np.linspace(y_min, y_max, ny+1)  # Create y values
z_vertices = np.linspace(z_min, z_max, nz+1)  # Create z values

# Use meshgrid to create the 3D vertices grid
XV, YV, ZV = np.meshgrid(x_vertices, y_vertices, z_vertices)

# Create the cell centers
x_centers = (x_vertices[:-1]+x_vertices[1:]) / 2.0  # Create x values
y_centers = (y_vertices[:-1]+y_vertices[1:]) / 2.0  # Create y values
z_centers = (z_vertices[:-1]+z_vertices[1:]) / 2.0  # Create z values

# Use meshgrid to create the 3D grid
X, Y, Z = np.meshgrid(x_centers, y_centers, z_centers)

# Calculate the step sizes in x, y and z directions
dx = x_centers[1] - x_centers[0]
dy = y_centers[1] - y_centers[0]
dz = z_centers[1] - z_centers[0]

# Calculate the wavenumbers in x, y and z directions
kx = 2 * np.pi * fftfreq(nx, dx)
ky = 2 * np.pi * fftfreq(ny, dy)
kz = 2 * np.pi * rfftfreq(nz, dz)

# Use meshgrid to create the 2D grid
KX, KY, KZ = np.meshgrid(kx, ky, kz)
K2 = KX**2 + KY**2 + KZ**2
K = np.sqrt(K2)
INVK2 = np.where(K2==0, 0, 1.0/K2)
KNORM = np.sqrt((KX/np.max(KX))**2 + (KY/np.max(KY))**2 + (KZ/np.max(KZ))**2)
SPEC_FILTER = np.exp(-23.6 * 1024 * (KNORM - 2.0/3.0)**4.)
SPEC_FILTER[KNORM <= 2.0/3.0] = 1

/var/folders/7s/tn4_jx8158vf7rpdxlfyncr40000gn/T/ipykernel_47346/2643438881.py:50: RuntimeWarning: divide by zero encountered in divide
  INVK2 = np.where(K2==0, 0, 1.0/K2)


In [98]:
def create_vtk_image_data(vouts):
    # Create vtkImageData
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(nx+1, ny+1, nz+1)
    imageData.SetSpacing(dx, dy, dz)
    imageData.SetOrigin(x_min, y_min, z_min)
    
    # Create temperature data array
    for key in vouts:
        A = np.copy(vouts[key].ravel(order='F').reshape((-1, 3)))
        tempArray = vtk.vtkDoubleArray()
        tempArray.SetName(key)
        tempArray.SetNumberOfComponents(3)
        tempArray.SetNumberOfTuples(nx * ny * nz)
        tempArray.SetArray(A, nx * ny * nz * 3, 1)
        imageData.GetCellData().SetScalars(tempArray)
    
    return imageData

def write_vtk_image_data(imageData, filename):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(imageData)
    writer.SetDataMode(vtk.vtkXMLWriter.Binary)
    writer.Write()

In [99]:
def rfftnv(F):
    # F has a shape of (3, ny, nx, nz)
    return np.array([
            rfftn(F[0]),
            rfftn(F[1]),
            rfftn(F[2])
        ])
def irfftnv(F):
    # F has a shape of (3, ny, nx, nz)
    return np.array([
            irfftn(F[0]),
            irfftn(F[1]),
            irfftn(F[2])
        ])

def curl_3d(F, fft=False):
    if fft:
        return np.array([
            1j*KY*F[2] - 1j*KZ*F[1],
            1j*KZ*F[0] - 1j*KX*F[2],
            1j*KX*F[1] - 1j*KY*F[0]
        ])
    else:
        return np.array([
            irfftn(1j*KY*F[2] - 1j*KZ*F[1]),
            irfftn(1j*KZ*F[0] - 1j*KX*F[2]),
            irfftn(1j*KX*F[1] - 1j*KY*F[0])
        ])

def div_3d(U):
        F = rfftnv(U)
        return np.sum(irfftn(1j*KX*F[0] + 1j*KY*F[1] + 1j*KZ*F[2]))


OMEGAH = np.array([rfftn(brownian_noise(ny, nx, nz)),rfftn(brownian_noise(ny, nx, nz)),rfftn(brownian_noise(ny, nx, nz))])
OMEGAH *= SPEC_FILTER
OMEGAH /= np.sqrt(0.5*np.sum(curl_3d(OMEGAH * INVK2)**2)*dx*dy*dz)
OMEGAH *= 0.1

PSIHAT = OMEGAH * INVK2
U = curl_3d(PSIHAT)
E0 = 0.5*np.sum(U**2)*dx*dy*dz

nu = 1e-4
nt = 101
tf = 10
t = 0
CFL = 0.5
dt = CFL/max(np.max(np.abs(U[0]/dx)),np.max(np.abs(U[1]/dy)),np.max(np.abs(U[2]/dz)))
nt_sub = int(np.ceil(tf/nt/dt))
print(dt, nt_sub, E0, np.max(U))

0.021763219080240693 5 0.009999999999999997 0.35568998583783473


In [100]:
def N(OMEGAH):
    OMEGA = irfftnv(OMEGAH)
    PSIHAT = OMEGAH * INVK2
    UH = curl_3d(PSIHAT, fft=True)
    U = irfftnv(UH)
    NONL = np.zeros_like(OMEGAH)
    for i in range(3):
        NONL[i] = rfftn(
              OMEGA[0]*irfftn(1j*KX*UH[i])
            + OMEGA[1]*irfftn(1j*KY*UH[i])
            + OMEGA[2]*irfftn(1j*KZ*UH[i])
            - U[0]*irfftn(1j*KX*OMEGAH[i]) 
            - U[1]*irfftn(1j*KY*OMEGAH[i]) 
            - U[2]*irfftn(1j*KZ*OMEGAH[i])
            ) * SPEC_FILTER
        NONL[i] += nu * (-K2 * OMEGAH[i])
    return NONL

for i in range(nt):
    for _ in range(nt_sub):
        k1 = dt * N(OMEGAH)
        k2 = dt * N(OMEGAH+1.0/2.0*k1)
        k3 = dt * N(OMEGAH+1.0/2.0*k2)
        k4 = dt * N(OMEGAH+k3)
        OMEGAH += 1.0/6.0*(k1 + 2*k2 + 2*k3 + k4)
        t += dt

    PSIHAT = OMEGAH * INVK2
    U = curl_3d(PSIHAT)
    OMEGA = irfftnv(OMEGAH)
    E = 0.5*np.sum(U**2)*dx*dy*dz
    CFL = dt*max(np.max(np.abs(U[0]/dx)),np.max(np.abs(U[1]/dy)),np.max(np.abs(U[2]/dz)))
    print(f"{i}, {t:.2f}, {CFL:.2f}, {E/E0:.4f}, {div_3d(U)}")

    imageData = create_vtk_image_data({"U": U})
    filename = f"results/vort_transport_3d_{i:03d}.vti"
    write_vtk_image_data(imageData, filename)


0, 0.11, 0.48, 0.9921, 5.553580772755923e-28
1, 0.22, 0.46, 1.0052, 1.464125840090198e-27
2, 0.33, 0.46, 1.0332, -1.464125840090198e-27
3, 0.44, 0.44, 1.0726, -2.3224065049706588e-27
4, 0.54, 0.46, 1.1212, 1.0602290566170399e-27
5, 0.65, 0.48, 1.1771, -1.868022623563356e-27
6, 0.76, 0.52, 1.2380, -1.464125840090198e-27
7, 0.87, 0.52, 1.3021, -1.5146129380243427e-28
8, 0.98, 0.53, 1.3681, -3.6350710512584224e-27
9, 1.09, 0.54, 1.4347, 6.058451752097371e-28
10, 1.20, 0.55, 1.5008, 6.563322731438818e-28
11, 1.31, 0.60, 1.5652, -2.0194839173657902e-28
12, 1.41, 0.63, 1.6268, -9.592548607487504e-28
13, 1.52, 0.64, 1.6849, -8.077935669463161e-28
14, 1.63, 0.67, 1.7389, -2.0194839173657902e-28
15, 1.74, 0.71, 1.7898, 2.271919407036514e-27
16, 1.85, 0.71, 1.8387, -8.077935669463161e-28
17, 1.96, 0.72, 1.8863, -3.534096855390133e-28
18, 2.07, 0.73, 1.9321, 1.6155871338926322e-27
19, 2.18, 0.72, 1.9745, -5.048709793414476e-29
20, 2.29, 0.73, 2.0122, 2.3224065049706588e-27
21, 2.39, 0.73, 2.0442,